In [2]:
import heavylight
import pandas as pd
import numpy as np

# A Vectorised Actuarial Contingency Model

In [3]:
class Life(heavylight.Model):
    time_step = 1/12
    def num_pols_if(self, t):
        if t == 0:
            return self.data["initial_policies"]
        else:
            return self.num_pols_if(t-1) - self.num_deaths(t)
    
    def num_deaths(self, t):
        if t == 0:
            return 0
        else:
            return self.num_pols_if(t) * self.q_x_m(t)
    
    def age(self, t):
        """age at time t"""
        if t == 0:
            return self.data["initial_age"] # floating point
        else:
            return self.age(t-1) + Life.time_step
    
    def age_rounded(self, t):
        return np.round(self.age(t))
    
    def q_x_m(self, t):
        """monthly mortality rate"""
        return self.q_x(t) ** (Life.time_step)
    
    def q_x(self, t):
        """annual mortality rate"""
        return self.basis["mortality"].series.loc[self.age_rounded(t)]

In [4]:
ages = np.array([32+1/12, 42, 25+7/12])
ages

array([32.08333333, 42.        , 25.58333333])

In [5]:
ages ** 1/12

array([2.67361111, 3.5       , 2.13194444])

In [6]:
mortality = heavylight.Table.from_csv("sample_q_x_table.csv")

In [7]:
mortality.series.loc[np.array([20, 21])]

x
20    0.141392
21    0.146911
Name: value, dtype: float64

In [8]:
basis = {"mortality": mortality}

In [9]:
num_model_points = 3
data = {
    "initial_policies": np.ones(num_model_points),
    "initial_age" : np.array([32+1/12, 42, 25+7/12]),
    }

In [10]:
model = Life(data=data, basis=basis)

In [11]:
model.num_pols_if(2)

: 

: 